In [16]:
# Please install OpenAI SDK first: `pip3 install openai`
# !pip install -U openai
import json
from openai import OpenAI
import pandas as pd
import torch
# Check if the current `accelerator <https://pytorch.org/docs/stable/torch.html#accelerators>`__is available, and if not, use the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

def generate_response(user_prompt, API_KEY):
    client = OpenAI(api_key=API_KEY, base_url="https://api.deepseek.com")
    system_prompt = """
    The user will provide some social media post text, parse and output in JSON format, "1" if the post is toxic, and "0" if not toxic. 
    
    EXAMPLE INPUT: 
    COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
    
    EXAMPLE JSON OUTPUT:
    {
        "cyberbullying": 1
    }
    """
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        response_format={
            'type': 'json_object'
        },
        stream=False
    )
    
    return json.loads(response.choices[0].message.content)

def generate_response_multiclass(user_prompt, API_KEY):
    client = OpenAI(api_key=API_KEY, base_url="https://api.deepseek.com")
    system_prompt = """
    The user will provide some social media post text, parse and output in JSON format, multi-label classify the post for each of the following categories: toxic, severe toxic, obscene, threat, insult, identity hate. 
    
    EXAMPLE INPUT: 
    COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
    
    EXAMPLE JSON OUTPUT:
    {
        "toxic": 1,
        "severe toxic": 1,
        "obscene": 1,
        "threat": 0,
        "insult": 1,
        "identity hate": 0
    }
    """
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        response_format={
            'type': 'json_object'
        },
        stream=False
    )
    
    return json.loads(response.choices[0].message.content)

def call_all(list_of_comments, dpsk_func, API_KEY):
    """function to run the deepseek call on all the text data and output a json file"""
    json_outputs = []
    for i in list_of_comments:
        json_outputs.append(dpsk_func(i, API_KEY))
    # Convert the list into a JSON array
    json_array = json.dumps(json_outputs, indent=4)
    # Save the json array as a file
    with open('cyberbullying.json', 'w') as file:
        file.write(json_array)

Using cpu device


In [17]:
# Example usage LOAD THE DATASET HERE

# Load and drop missing values
df = pd.read_excel("train.xlsx")
df = df.dropna(subset=["comment_text", "cyberbullying"])

# Ensure types
df["comment_text"] = df["comment_text"].astype(str)
df["cyberbullying"] = df["cyberbullying"].astype(int)

# Extra safety: remove non-stringy things
def is_valid_text(t):
    return isinstance(t, str) and len(t.strip()) > 0

df = df[df["comment_text"].apply(is_valid_text)]

comments = df["comment_text"].tolist()
comments = comments[:3] # for testing

In [24]:
API_KEY = "PUT API KEY HERE"
call_all(comments, generate_response_multiclass, API_KEY)

Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)
Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.
